# Additional Modelling Challenges

## N-Queens

The N-Queens problem is a classic combinatorial problem that involves placing `n` chess queens on an `n x n` chess board so that no two queens threaten each other.

This means that no two queens can be in the same row, the same column, or the same diagonal.

The goal is to find all possible arrangements of the queens on the board.

In [1]:
import cpmpy as cp

# Parameters
n = 4

# Decision variables
board = cp.intvar(0, 1, shape=(n, n))  # 0 = no queen, 1 = queen

model = cp.Model()

# Constraints
# N queens
model += board.sum() == n

for row in range(n):
    for col in range(n):
        for other_pos in range(n):
            # no two queens in the same row
            if other_pos != row:
                model += (board[row, col] == 1).implies(board[other_pos, col] == 0)
            # no two queens in the same column
            if other_pos != col:
                model += (board[row, col] == 1).implies(board[row, other_pos] == 0)

        for incr in range(-n+1, n):
            # no two queens on the same diagonal
            if incr != 0:
                diag1_row, diag1_col = row + incr, col + incr
                diag2_row, diag2_col = row + incr, col - incr
                
                if 0 <= diag1_row < n and 0 <= diag1_col < n:
                    model += (board[row, col] == 1).implies(board[diag1_row, diag1_col] == 0)
                if 0 <= diag2_row < n and 0 <= diag2_col < n:
                    model += (board[row, col] == 1).implies(board[diag2_row, diag2_col] == 0)

model.solveAll(display=board)

[[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]]
[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]


2

In [2]:
# break symmetry by requiring first queen to be in first half of first row
model += board[0, :n//2].sum() >= 1

model.solveAll(display=board)

[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]


1

Backtracking Search:

In [3]:
import numpy as np


def conflicts(row: int, col: int, board: np.ndarray) -> bool:
    if board.sum() == 0:
        return False

    n = len(board)

    if board[row, :].sum() != 0:
        return True
    if board[:, col].sum() != 0:
        return True

    lu_diag_row, lu_diag_col = row - 1, col - 1
    while lu_diag_row >= 0 and lu_diag_col >= 0:
        if board[lu_diag_row, lu_diag_col] != 0:
            return True
        lu_diag_row -= 1
        lu_diag_col -= 1

    ll_diag_row, ll_diag_col = row + 1, col - 1
    while ll_diag_row < n and ll_diag_col >= 0:
        if board[ll_diag_row, ll_diag_col] != 0:
            return True
        ll_diag_row += 1
        ll_diag_col -= 1

    ru_diag_row, ru_diag_col = row - 1, col + 1
    while ru_diag_row >= 0 and ru_diag_col < n:
        if board[ru_diag_row, ru_diag_col] != 0:
            return True
        ru_diag_row -= 1
        ru_diag_col += 1

    rl_diag_row, rl_diag_col = row + 1, col + 1
    while rl_diag_row < n and rl_diag_col < n:
        if board[rl_diag_row, rl_diag_col] != 0:
            return True
        rl_diag_row += 1
        rl_diag_col += 1

    return False

In [4]:
from typing import Optional, Sequence, Tuple


def n_queens(n: int, *, board: Optional[np.ndarray] = None, frontier: Optional[Sequence[Tuple[int, int]]] = None) -> np.ndarray:
    if board is None:
        board = np.zeros((n, n))
    elif len(board) != n:
        raise ValueError(f"{len(board) = } != {n = }")
    if not frontier:
        frontier = [(n-i-1, n-j-1) for i in range(n) for j in range(n)]

    for row, col in frontier:
        if not conflicts(row, col, board):
            board[row, col] = 1

            new_frontier = [(r, c) for r, c in frontier if r != row and c != col]
            if (res := n_queens(n, board=board, frontier=new_frontier)).sum() == n:
                return res

            board[row, col] = 0

    return board


print(n_queens(10))

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
